In [1]:
import tensorflow as tf 
from tensorflow import keras
import keras.backend as K 
from keras import layers
from keras.utils.image_utils import array_to_img 
import matplotlib.pyplot as plt 
import os 
import numpy as np
import random


2023-01-06 09:19:40.537813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 09:19:40.684960: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-06 09:19:41.259198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64${LD_LIBRARY_PATH:+:/usr/lib/cuda/include:/usr/lib/cuda/lib64:}
2023-01-06 09:19:41.259263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [2]:
global HEIGHT, WIDTH,CHANNEL,LATENT_DIM
HEIGHT = 360; WIDTH = 640; CHANNEL =3 ; LATENT_DIM = 32; 
BATCH_SIZE = 2; LR_INIT = 1e-3; THRESHOLD = 1.5;
 

In [3]:
def AE_Encoder():
    encoder_inputs = keras.Input(name="Encoder_inputs",shape = (HEIGHT,WIDTH,CHANNEL))
    x = layers.Conv2D(32,(5,5),strides=(2,2),padding ="same",name= "conv_1")(encoder_inputs)
    x = layers.Conv2D(64,(3,3),strides=(2,2),padding ="same",name= "conv_2")(x)
    x = layers.Conv2D(128,(3,3),strides=(2,2),padding ="same",name= "conv_3")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation="relu",name="encoder_dense2")(x)   
    x = layers.Dense(LATENT_DIM,name= "hyper_dense")(x)
    return keras.Model(inputs =encoder_inputs,outputs =x,name="ae_encoder")


def AE_Decoder():
  latent_inputs= layers.Input(shape=(LATENT_DIM,),name="latent_input")
  x = layers.Dense(128,activation="relu",name="decoder_dense1")(latent_inputs)
  x= layers.Dense(45*80*128,activation="relu",name="decoder_dense3")(x)
  x = layers.Reshape((45,80,128))(x)
  x = layers.Conv2DTranspose(128,(3,3),strides=(2,2),padding="same",activation="relu",name="decoder_T2D_1")(x)
  x = layers.Conv2DTranspose(64,(3,3),strides=(2,2),padding="same",activation="relu",name="decoder_T2D_2")(x)
  x = layers.Conv2DTranspose(32,(3,3),strides=(2,2),padding="same",activation="relu",name="decoder_T2D_3")(x)
  decoder_outputs = layers.Conv2D(CHANNEL,(3,3),strides=(1,1),padding="same",name="aede_out")(x)
  return keras.Model(latent_inputs,decoder_outputs,name="ae_decoder")


## Try larger strides and kernel and more CNN layer to make sure when it is flatten the parameter wont be too much !
def Encoder():
    encoder_inputs = keras.Input(name="Encoder_inputs",shape = (HEIGHT,WIDTH,CHANNEL))
    x = layers.Conv2D(32,(5,5),strides=(2,2),padding ="same",name= "conv_1")(encoder_inputs)
    x = layers.Conv2D(64,(3,3),strides=(2,2),padding ="same",name= "conv_2")(x)
    x = layers.Conv2D(128,(3,3),strides=(2,2),padding ="same",name= "conv_3")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation="relu",name="encoder_dense2")(x)    
    x = layers.Dense(LATENT_DIM,name= "hyper_dense")(x)
    return keras.Model(inputs =encoder_inputs,outputs =x,name="encoder")

def feature_extractor():
    input_layer = layers.Input(name="extractor_input",shape=(HEIGHT,WIDTH,CHANNEL))
    x = layers.Conv2D(32,(5,5),strides=(2,2),padding ="same",name= "extractor_conv_1",kernel_regularizer="l2")(input_layer)
    x = layers.LeakyReLU(name="extractor_leaky_1")(x)
    x = layers.BatchNormalization(name="extractor_norm_1")(x)
    x = layers.Conv2D(64,(3,3),strides=(2,2),padding ="same",name= "extractor_conv_2",kernel_regularizer="l2")(x)
    x = layers.LeakyReLU(name="extractor_leaky_2")(x)
    x = layers.BatchNormalization(name="extractor_norm_2")(x)
    x = layers.Conv2D(128,(3,3),strides=(2,2),padding ="same",name= "extractor_conv_3",kernel_regularizer="l2")(x)
    x = layers.LeakyReLU(name="extractor_leaky_3")(x)
    x = layers.BatchNormalization(name="extractor_norm_3")(x)
    return keras.Model(input_layer,x,name="feature_extractor")

def Descriminator(fe):
  input_layer = layers.Input(shape=(HEIGHT,WIDTH,CHANNEL))
  x = fe(input_layer)
  x = layers.GlobalAveragePooling2D(name="glb_avg")(x)
    
  x = layers.Dense(1,activation = "sigmoid",name="d_output")(x)
  return keras.Model(input_layer,x,name="discriminator")



In [4]:
K.clear_session
global encod,decod,Encoder,d,AE
encod = AE_Encoder()
decod = AE_Decoder()
Encoder = Encoder()
feature_extractor = feature_extractor()
d = Descriminator(feature_extractor)
AE = keras.models.Sequential(name="AutoEncoder")
AE.add(encod)
AE.add(decod)

2023-01-06 09:19:42.493603: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 09:19:42.498484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 09:19:42.498907: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 09:19:42.499953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
d.compile(optimizer = keras.optimizers.Adam(learning_rate = LR_INIT),loss=keras.losses.BinaryCrossentropy())

In [6]:
# Adv loss is the way discriminator stimulate the ae to learn
class AdvLoss(keras.layers.Layer):
    def __init__(self,**kwargs):
        super(AdvLoss,self).__init__(**kwargs)
    @tf.function
    def call(self,x,mask=None):
        feature_extractor = d.get_layer("feature_extractor")
        ori_feature = feature_extractor(x[0])
        gan_feature = feature_extractor(x[1])
        return tf.reduce_mean(
                            tf.square( ori_feature - gan_feature  ))
    def compute_output_shape(self,input_shape):
        return (input_shape[0][0],3)

# CntLoss is the way of ae itself to learn
class CntLoss(keras.layers.Layer):
    def __init__(self,**kwargs) :
        super(CntLoss,self).__init__(**kwargs)
    @tf.function
    def call(self,x,mask=None):
        ori = x[0]
        gan = x[1]
        cnt_loss = tf.reduce_mean(
                        tf.reduce_sum(
                            keras.losses.MAE(ori,gan),
                            axis = (1,2)))
        return cnt_loss
    def compute_output_shape(self,input_shape):
        return (input_shape[0][0],3)

class EncLoss(keras.layers.Layer):
    def __init__(self,**kwargs):
        super(EncLoss,self).__init__(**kwargs)
    @tf.function
    def call(self,x,mask=None):
        # ae = ae_trainer.get_layer("AutoEncoder")
        encod = AE.get_layer("ae_encoder")
        ori = x[0]
        gan = x[1]
        enc_loss = tf.reduce_sum(
                      keras.losses.MSE( Encoder(gan),encod(ori)))
        return enc_loss
    def compute_output_shape(self,input_shape):
        return (input_shape[0][0],3)

def loss(y_true,y_pred):
    return y_pred

In [7]:
input_layer = layers.Input(name = "input",shape=(HEIGHT,WIDTH,CHANNEL))
gan = AE(input_layer)

adv_loss = AdvLoss(name = "adv_loss")([input_layer,gan])
cnt_loss = CntLoss(name="cnt_loss")([input_layer,gan])
enc_loss = EncLoss(name="enc_loss")([input_layer,gan])

ae_trainer = keras.models.Model(input_layer,[adv_loss,cnt_loss,enc_loss],name="ae_trainer")
losses = {"adv_loss":loss,"cnt_loss":loss,"enc_loss":loss}
lossWeights = {"cnt_loss":40.0,"adv_loss":1,"enc_loss":1}

In [8]:
ae_trainer.compile(optimizer=keras.optimizers.Adam(learning_rate = LR_INIT),
                   loss= losses,
                   loss_weights=lossWeights,
                   )

In [9]:
dataset = tf.data.Dataset.load("tfdata")
dataset = dataset.shuffle(100)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat(2)


In [10]:
import datetime
print(datetime.datetime.now().date())

2023-01-06


In [17]:
import os

def LRschedule(Model):
  lr  = Model.optimizer.learning_rate
  # print(f"lr of {Model.name}:{Model.optimizer.learning_rate.numpy()}\t")
  New_lr = lr * 0.001**(1/1500)
  K.set_value(Model.optimizer.learning_rate,New_lr)
  print(f"Deacyed :{Model.optimizer.learning_rate.numpy()}\n")

def SaveModel(AE,Encoder,d,iter):
  path_string = "/content/drive/MyDrive/Monument/{}GANAE_128/{}epoch".format(datetime.datetime.now().date(),iter)
  path = os.path.join(path_string)
  if os.path.exists(path) is False:
    os.makedirs(path)
    print(f"Making dir {path}")
  AE.save(path +"/AE.h5")
  AE.save_weights(path+"/AE_weights.h5")
  Encoder.save(path+"/Encoder.h5")
  Encoder.save_weights(path+"/Encoder_weights.h5")
  d.save(path+"/Discriminator.h5")
  d.save_weights(path+"/Discriminator_weights.h5")
  print("Models at {} epoch saved!\n".format(iter))

def SaveHistory(d_loss_History,g_loss_History,iter):
      g_loss_history = np.array(g_loss_History)
      d_loss_history = np.array(d_loss_History)
      import pandas as pd 
      g_loss_dict = {"cnt_loss":g_loss_history[:,0],"adv_loss":g_loss_history[:,1],"enc_loss":g_loss_history[:,2],"loss_unknown":g_loss_history[:,3]}
      d_loss_dict = {"d_loss":d_loss_history} 
      df_g = pd.DataFrame(g_loss_dict)
      df_d = pd.DataFrame(d_loss_dict)
      path_string = "/content/drive/MyDrive/Monument/{}GANAE_128/{}epoch".format(datetime.datetime.now().date(),iter)
      path = os.path.join(path_string)
      if os.path.exists(path) is False:
        os.makedirs(path)
        print(f"Making dir {path}")
      df_g.to_csv(os.path.join(path,"g_loss{}.csv".format(iter)))
      df_d.to_csv(os.path.join(path,"d_loss{}.csv".format(iter)))
      print("Histories at {} epoch saved!\n".format(iter))

In [12]:
d_loss_History = []
g_loss_History = []
niter = 1
for i in range(niter+1):
  x = dataset.take(1)
  d.trainable = True
  fake_x = AE.predict(x,verbose = 0)
  xt = iter(x).next(); xt = xt.numpy()
  # print(xt)
  print(fake_x.shape)
  d_x = np.concatenate([xt,fake_x],axis=0)
  d_y = np.concatenate([np.zeros(len(xt)),np.ones(len(fake_x))],axis=0)

  d_loss = d.train_on_batch(d_x,d_y)
  d.trainable = False
  
  ########### Training Generator ##########
  g_loss = ae_trainer.train_on_batch(xt,xt)

  ########### Appending Loss ###############  
  d_loss_History.append(d_loss)
  g_loss_History.append(g_loss)
  
  if i%50 == 0 and i != 0:
        LRschedule(d)
        LRschedule(ae_trainer)
  ########### Loss Report every 10epochs ##########
  if i%10 ==0:
        print(f"Epoch:{i},g_loss:{g_loss},d_loss={d_loss}")
        print("d is trainable{}".format(d.trainable))

  ########### Model Callbacks every 100epochs ##########
  # if i == 50: 
  #     SaveModel(AE,Encoder,d,iter=i)
  #     SaveHistory(d_loss_History,g_loss_History,iter=i)
  if i%100==0 and i != 0:
      SaveModel(AE,Encoder,d,iter=i)
      SaveHistory(d_loss_History,g_loss_History,iter=i)


####################################################################

2023-01-06 09:19:44.598674: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


(2, 360, 640, 3)
(2, 360, 640, 3)
